In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pymysql
import pymssql
import sys
import os
from functools import reduce

In [ ]:
sys.path.insert(0, os.getcwd())
sys.path.insert(0, '/home/vishal/refactoring_pipeline')

In [ ]:
import query as q
import extract
import preprocess as prep
import utilities as util
import model_compare as mod_com
from sklearn.metrics import confusion_matrix as cm
import EDA as eda

bank_app - loanid, entered_date, bv decisions

predicon_model - loan_id, entered_date, count of scored applicants, #positive

loans - loanid, lead_time_added, #lender approved

## In a Nutshell

### Extracting Data

In [ ]:
bank_app = extract.extract_bankapp("'2020-06-09'")

In [ ]:
model = extract.extract_model()

In [ ]:
loan_history = extract.extract_loan_history("'2020-06-09'")

### Modifying Data

In [ ]:
modified_bank_app = prep.preprocess_bankapp_db(bank_app)

In [ ]:
modified_model_scores = prep.preprocess_model_db(model)

In [ ]:
modified_loan_history = prep.preprocess_loan_history_db(loan_history)

In [ ]:
is_BV_uncertain_approved = modified_bank_app['agent_decision'].isin(['Bank Validation Uncertain', 'Bank Validation Approved'])
modified_bank_app = modified_bank_app[is_BV_uncertain_approved]

### Merging all the modified datasets and generating model kpi metrics

In [ ]:
kpi_func = kpi_count.get_kpi(md_bnk, md_mod, md_fl)

In [ ]:
kpi_func

### Analysing loanids which were not lender approved but our model approved

In [ ]:
dfs = [modified_bank_app, modified_model_scores, modified_loan_history]
merged_db = reduce(lambda left, right : pd.merge(left, right, on = 'LoanId', how = 'left'), dfs)

In [ ]:
merged_db['Decision'].fillna(-1, inplace = True)

In [ ]:
is_lender_approved_present = merged_db['LenderApproved'].notnull()
cm(merged_db['LenderApproved'][is_lender_approved_present], merged_db['Decision'][is_lender_approved_present])

In [ ]:
lender_reject_model_accept = set(merged_db[(merged_db['LenderApproved'] == 0) & (merged_db['Decision'] == 1)]['LoanId'].values)

In [ ]:
df_lender_reject_model_accept = merged_db[merged_db['LoanId'].isin(lender_reject_model_accept)][['underwriting_final_decision', 'agent_decision']]

In [ ]:
df_lender_reject_model_accept['underwriting_final_decision'].value_counts()

In [ ]:
eda.countplot_categorical_columns(df_lender_reject_model_accept, cols = ['underwriting_final_decision'])

In [ ]:
pd.crosstab(merged_db['LenderApproved'][is_lender_approved_present], merged_db['Decision'][is_lender_approved_present])